# Fintech Challenge

In recent weeks, a fintech company has identified a significant drop of the pass rate in KYC procedure. KYC procedure consists of the following two steps: 

* Document check: To verify that the photo ID is valid and authentic;
* Facial Similarity check: To verify that the face in the picture is the same with that on the submitted ID.

I have been granted access to the results (DOCUMENT REPORT -DR- and FACIAL SIMILARITY -FS-) obtained by the two checks for 142.724 users and a total of 176.404 attempts from May to October 2017.

In both the DR and FS data sets, each row represents an attempt (unique attempt_id) to pass the checks. 

Since the pass rate is defined as the number of customers who pass both the KYC process divided by the number of customers who attempt the process, it makes sense to look at a merged dataframe that offers the results and features of both checks - althought first I have had to make sure that the data frames can be properly joined -.

In order to understand the magnitude of the issue, I compute the pass rate per week (see .png file, result of plotting the variable 'grouped_results' defined in the code below): 

<img src="files/pass_rate.png">


Clearly, it has been going down since its peak in week 24 where 95,5% of attempts were succesful. While I am not familiar with the business, I believe that a pass rate below 90% is not acceptable. In order to understand the issues that have caused the decline, I will split the data set in two: 

* **dr_fs_27**: contains all attempts until week 27. 
* **dr_fs_44**: contains all attempts from week 28 to week 44. 

By doing the same exploratory work on both data sets and comparing its results, hopefully I'll notice what is the root cause of the decline. 

### How to use this document 

The following report contains both the code and the qualitative analysis of the results I obtain, so it combines both code and markdown snippets. The best way to use it is by running all cells. The code will print some intermediate results.  


In [49]:
import pandas as pd
import numpy as np

facial_reports = pd.read_csv('facial_similarity_reports.csv')
docs_reports = pd.read_csv('doc_reports.csv')

In [50]:
#--------------------
#SUPPORT FUNCTIONS
#--------------------

#Obtain week from date columns into new feature
def parse_date(df,date_feature,new_date_feature):
    df[date_feature] =  pd.to_datetime(df[date_feature], format='%Y-%m-%dT%H:%M:%S')
    df[new_date_feature] = df[date_feature].dt.week
    return df

#Parse dic-type feature to multiple features and concat datasets
def parse_dic(df,dic_type_feature):
    df[dic_type_feature] = df[dic_type_feature].apply(lambda x : dict(eval(x)) )
    parsed_df = df[dic_type_feature].apply(pd.Series )
    df = pd.concat([df, parsed_df], axis=1).drop(dic_type_feature, axis=1)
    return df

#Return df indexes of observations of feature var that appear more than num_filt times. 
def rmv_mult_values(df,var,num_filt):
    df_s = df.groupby(var).filter(lambda x: len(x) > num_filt)
    df_s_indx = df_s.index.values
    return df_s, df_s_indx

#Check left and inner join of dataframe merges give same results. If check is sucessful, returns merged dataframe.
def check_join(df1,df2,join_feature):
    df3 = pd.merge(df1, df2, on=join_feature, how='inner')
    df4 = pd.merge(df1, df2, on=join_feature, how='left')
    if (df3.shape == df4.shape):
        return df3
    else:
        pass
    
#Adds prefix to data sets before joining them.
def prep_features_join(df,feature_list,prefix): 
    pf_feature_list =[prefix + feature for feature in feature_list]
    pf_replace = dict(zip(feature_list, pf_feature_list))
    df = df.rename(columns = pf_replace)
    return df


#Return data set df with new feature --> # attempts per user 
def count_attempts(df,new_feature):
    uid_c = df['user_id'].value_counts().rename(new_feature)
    df_counts = df.merge(uid_c.to_frame(),left_on='user_id', right_index=True)
    return df_counts

In [51]:
#Join DR and FS datasets. 
docs_reports = parse_dic(docs_reports,'properties')

#Add DR prefix to its features (only to those that are unique to DR data set)
dr_feature_list = ['result', 'visual_authenticity_result',
       'image_integrity_result', 'face_detection_result',
       'image_quality_result', 'created_at', 'supported_document_result',
       'conclusive_document_quality_result', 'colour_picture_result',
       'data_validation_result', 'data_consistency_result',
       'data_comparison_result', 'police_record_result',
       'compromised_document_result', 'properties', 'sub_result']

fs_feature_list = ['result', 'face_comparison_result',
       'created_at', 'facial_image_integrity_result',
       'visual_authenticity_result', 'properties']

docs_reports = prep_features_join(docs_reports,dr_feature_list,'DR_')
facial_reports = prep_features_join(facial_reports,fs_feature_list,'FS_')

#perform merge of data sets
merged_dr_fs = pd.merge(docs_reports, facial_reports, on='attempt_id', how='inner')

#drop redundant features
merged_dr_fs = merged_dr_fs.drop(['user_id_y', 'Unnamed: 0_y'], axis=1)

#compute week of the attempt
merged_dr_fs = parse_date(merged_dr_fs,'DR_created_at','week_attempt')

#compute feature pass_KYC (1 if succesful, 0 if not)
conditions = [(merged_dr_fs['DR_result'] == 'clear') & (merged_dr_fs['FS_result'] == 'clear')]
val = [1]
merged_dr_fs['pass_KYC'] = np.select(conditions, val, default=0)
pass_rate_grouped = merged_dr_fs.groupby('week_attempt').agg({'pass_KYC': ['sum','count']})

#group results by partial checks DOCUMENT and FACIAL
grouped_results = merged_dr_fs.groupby(['DR_result', 'FS_result']).count()['attempt_id']

In [52]:
#Compute two data sets
# dr_fs_27 --> attempts before week 28 
# dr_fs_44 --> attempts after week 28 
dr_fs_27 = merged_dr_fs.loc[merged_dr_fs['week_attempt'] <= 27]
dr_fs_44 = merged_dr_fs.loc[merged_dr_fs['week_attempt'] > 27]
print('..........................................................................')
print('SEGMENT WEEK 21 - 27 ATTEMPTS BY RESULT AND SUB_RESULT')
print('..........................................................................')
print(dr_fs_27.groupby(['DR_result', 'FS_result']).count()['attempt_id'])
print('..........................................................................')
print('SEGMENT WEEK 28 - 44 ATTEMPTS BY RESULT AND SUB_RESULT')
print('..........................................................................')
print(dr_fs_44.groupby(['DR_result', 'FS_result']).count()['attempt_id'])

..........................................................................
SEGMENT WEEK 21 - 27 ATTEMPTS BY RESULT AND SUB_RESULT
..........................................................................
DR_result  FS_result
clear      clear        15319
           consider       987
consider   clear          522
           consider        77
Name: attempt_id, dtype: int64
..........................................................................
SEGMENT WEEK 28 - 44 ATTEMPTS BY RESULT AND SUB_RESULT
..........................................................................
DR_result  FS_result
clear      clear        111570
           consider       4524
consider   clear         38074
           consider       5329
Name: attempt_id, dtype: int64


In light of these results, it looks like there's a fundamental shift as to why attempts fail between the two data sets. In the "healthy" data set, the fail rate (1 - pass_rate) is rougly 9%; of which 6% comes from passing the document check but not the facial similarity check. 

On the other hand, in the "damaged" data set, the driver of the fail rate is not a fail in the facial similarity check but in the document check. In particular, the fail rate is 30%, of which 24% comes from not passing the document check.  Clearly this is a significant issue that needs to be adressed. Rougly one fourth of new users cannot finish the on-boarding process correctly hence Revolut's user acquisition activity is damaged and so is its user experience. 

Given this new discovery, it makes sense to focus on the DR dataset. Hopefully, a thorough analysis of it will shed light on why there are so many failed attempts. 

As per the Veritas API documentation, DOCUMENT REPORT presents two differnet result features, namely: 
* result: The overall result of the check, can take two values - 'consider' / 'clear'
* sub_result, which indicates a more detailed result. It can take the following values: 
    * **clear**	If all underlying verifications pass, the overall sub result will be clear
    * **rejected**	If the report has returned information where the check cannot be processed further (poor quality image or an unsupported document).
    * **suspected**	If the document that is analysed is suspected to be fraudulent.
    * **caution** if any other underlying verifications fail but they don’t necessarily point to a fraudulent document (such as the name provided by the applicant doesn’t match the one on the document)
    
By segmenting the attempts of the 'damaged' data set  by result and sub_result, it looks like most unsuccesful attempts report 'caution' or 'rejected' in sub_result (15.469 and 26.071 failed attempts, respectively). It might be worth to analyse them separately.

Following with the API documentation,  the DOCUMENT REPORT is composed of data integrity, visual authenticity and police record checks. It checks the internal and external consistency of the most recent identity document provided by the applicant to identify potential discrepancies.

Although not stated explicitly, it looks like the result - feature 'result' -  depends on the results of simpler checks, referenced in the following features: 

* Visual authenticity
* Image integrity
    * Face detection
    * Image quality
    * Supported document
    * Conclusive document quality
    * Colour picture
* Data validation
* Data consistency
* Data comparison
* Police record
* Compromised document

A simple exploratory analysis shows that most of these features take one of the following values: NaN (empty field), 'clear' or 'consider'. Additionally, it looks like image integrity fields - image quality and supported document - also take the value 'unidentified'. 

The Veritas API does not seem to refer to the meaning of NaN values in these features. It would be worth exploring about these empty fields - and more importantly, investigate why they are not filled correctly.  Let's leave this for later. 

Perhaps what is more relevant is that in almost 40.000 attempts, the feature referring to the image integrity test was labelled as 'consider' , meaning it didn't pass the check. The image integrity check seems to depends on the partial checks regarding face detection, image quality, supported document, conclusive document quality and colour picture so it's worth investigating these further. 

At this point I have enough data to state a hypothesis - most of the decrease in failing rate is due to a failure in passing the image integrity sub check, which is part of the document check. 

In the 'damaged' data set, there seems to be a significant increase in 'rejected' attempts as per the value of the feature sub_result. According to the API documentation, the feature takes this value if the report has returned information where the check cannot be processed further (poor quality image or an unsupported document). This information relates very well to the hypothesis I stated above. 

In fact, after removing all rows in the damaged data set where the image integrity result is NOT 'consider', I end up with a set that does not contain any 'rejected' attempts (with some 'caution' and 'consider'), so it seems there is a direct cause effect between attempts with poor image integrity and those that end up in 'rejected'. Plus, since most of the failed attempts have sub_result 'rejected', it makes sense to focus those attempts first. 

In [53]:
#Segment attempts by result and sub_result - clearly there is a significant increase in 'rejected' results. 
print('..........................................................................')
print('WEEK 21 - 27 DATA SET: SEGMENT ATTEMPTS BY RESULT AND SUB_RESULT')
print('..........................................................................')
print(dr_fs_27.groupby(['DR_result','DR_sub_result']).count()['attempt_id'])
print('..........................................................................')
print('WEEK 28 - 44 DATA SET: SEGMENT ATTEMPTS BY RESULT AND SUB_RESULT')
print('..........................................................................')
print(dr_fs_44.groupby(['DR_result','DR_sub_result']).count()['attempt_id'])


f = ['DR_visual_authenticity_result',
       'DR_image_integrity_result', 'DR_data_validation_result',
       'DR_data_consistency_result', 'DR_data_comparison_result',
       'DR_police_record_result', 'DR_compromised_document_result']

for element in f: 
    print('..........................................................................')
    print('NUMBER OF ATTEMPTS GROUPED BY' +' ' + element)
    print('..........................................................................')
    print(dr_fs_44.astype(str).groupby([element]).count()['attempt_id'])

..........................................................................
WEEK 21 - 27 DATA SET: SEGMENT ATTEMPTS BY RESULT AND SUB_RESULT
..........................................................................
DR_result  DR_sub_result
clear      clear            16306
consider   caution            523
           rejected            19
           suspected           57
Name: attempt_id, dtype: int64
..........................................................................
WEEK 28 - 44 DATA SET: SEGMENT ATTEMPTS BY RESULT AND SUB_RESULT
..........................................................................
DR_result  DR_sub_result
clear      clear            116095
consider   caution           15469
           rejected          26071
           suspected          1863
Name: attempt_id, dtype: int64
..........................................................................
NUMBER OF ATTEMPTS GROUPED BY DR_visual_authenticity_result
...............................................

In [54]:
test_hypothesis = dr_fs_44.loc[dr_fs_44['DR_image_integrity_result'] != 'consider']
print('..........................................................................')
print('SEGMENT ATTEMPTS WHERE IMAGE_INTEGRITY <> CONSIDER;  BY RESULT AND SUB_RESULT')
print('..........................................................................')
print(test_hypothesis.groupby(['DR_result','DR_sub_result']).count()['attempt_id'])

..........................................................................
SEGMENT ATTEMPTS WHERE IMAGE_INTEGRITY <> CONSIDER;  BY RESULT AND SUB_RESULT
..........................................................................
DR_result  DR_sub_result
clear      clear            116095
consider   caution            2031
           suspected          1648
Name: attempt_id, dtype: int64


In [55]:
#DEFINE NEW DATA SET dr_fs_iif; FILTER merged_dr_fs TO KEEP ONLY ATTEPMTS WHERE image_integrity_result == 'consider' 

#Group attempts with image_integrity_result = 'consider' by week
#Clearly, the issue starts to appear in week 28. 
dr_fs_iif = merged_dr_fs.loc[merged_dr_fs['DR_image_integrity_result'] == 'consider']
print('..........................................................................')
print('NUMBER OF ATTEMPTS WITH FAILURE IN IMAGE INTEGRITY OVER THE TIME PERIOD')
print('..........................................................................')
print(dr_fs_iif.groupby('week_attempt').agg({'pass_KYC': ['count']}))

#Group attempts with image_integrity_result = 'consider' in the damaged data set
dr_fs_44_iif = dr_fs_44.loc[dr_fs_44['DR_image_integrity_result'] == 'consider']


#Analysis of image integrity sub features 
f_b = ['DR_image_quality_result',
       'DR_supported_document_result', 'DR_conclusive_document_quality_result',
       'DR_colour_picture_result']

for element in f_b: 
    print('..........................................................................')
    print('NUMBER OF ATTEMPTS GROUPED BY' +' ' + element)
    print('..........................................................................')
    print(dr_fs_44_iif.astype(str).groupby([element]).count()['attempt_id'])    

a = dr_fs_44_iif[dr_fs_44_iif['DR_conclusive_document_quality_result'].isnull()]
print(a.astype(str).groupby(['DR_image_quality_result']).count()['attempt_id'])

..........................................................................
NUMBER OF ATTEMPTS WITH FAILURE IN IMAGE INTEGRITY OVER THE TIME PERIOD
..........................................................................
             pass_KYC
                count
week_attempt         
23                  2
24                  2
25                  4
26                  7
27                  4
28                713
29                887
30               1055
31               1027
32               1009
33               1160
34               1128
35               1260
36               1219
37               1900
38               3003
39               3398
40               6169
41               6989
42               3056
43               4992
44                759
..........................................................................
NUMBER OF ATTEMPTS GROUPED BY DR_image_quality_result
..........................................................................
DR_image_quality_result


### Conclusions and further work 

On-boarding pass rate for new users has decreased significantly in recent weeks. From week 21 to 27, pass rate remained above 90%, and since week 28 has decreased bottoming at 55% in week 41. Comparing both data sets, it's clear there's a fundamental shift as to why attempts fail. In the "healthy" data set, the fail rate (1 - pass_rate) is rougly 9%; of which 6% comes from passing the document check but not the facial similarity check, whereas in the "damaged" data set, the driver of the fail rate is not a fail in the facial similarity check but in the document check. In particular, the fail rate is 30%, of which 24% comes from not passing the document check.

At a closer look, the document check's fail rate is around 27% (43.403 attempts). 60% of these attempts are labelled as rejected, 35% as caution and 5% as suspected.


Rejected attempts are due to an issue around the integrity of the picture of the document (picture of the passport, national ID or driving license) uploaded for the document check. In particular, it seems like the picture of the document is not read properly due to supposedly bad quality.  

Before exploring solutions, first I would manually, if possible, try to find further data that reinforces this conclusion. For instance, we could potentially explore the look&feel of some of the damaged pictures. This would allow us to have a better understanding of the problem. It could potentially help answer questions like: is it our software that is not working properly? Or is it the users that do not understand how should the picture be taken? Another piece of data that would reinforce this conclusion would be to look at the topic of tickets opened in customer support. 

In order to address this issue, there are few areas we could potentially take action on. Regarding product, we could review the on-boarding process and in-screen help to ensure the user is aware how should she take the picture. 
We could also potentially include some sort of photo-filter to improve its quality before sending it to Veritas for verification. 


### Further work 

While this analysis addresses the 60% of the failed attempts in Document check, I would continue my analysis to understand why there are so many attempts labelled as 'caution'. 